In [ ]:
from utils import *
from diffusers import StableDiffusionPipeline
import json
import torch

In [ ]:
# Load the model

model_id = "/root/annotated_textual_inversion/model/sd-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to("cuda")
pipe.safety_checker = lambda images, clip_input: (images, False)

In [ ]:
# Load the embedding

token = "<Monet>"
number = 2
weight_dir="/root/annotated_textual_inversion/embeds/Monet-TI-2000"
name="learned_embeds"

# for MTI
# token = load_MTI_embeds(pipe, model_id, token, number, weight_dir, name)

# for TI
pipe.load_textual_inversion(model_id, token, weight_name=os.path.join(weight_dir, f"{name}.bin"))

In [ ]:
# generate samples

# prompt = f"a good photo of a {token}"
prompt = f"a painting in the style of {token}"
output_dir = f"/root/annotated_textual_inversion/output/samples"
num_samples = 20
step = 50
guidance_scale = 8
generator = torch.Generator(device="cuda").manual_seed(0)

generate_sample(pipe, prompt, output_dir, num_samples, step, guidance_scale, generator)

In [ ]:
# validate

loss_fn = lpips.LPIPS(net='alex').cuda()
target_dir = "/root/annotated_textual_inversion/data/Monet/train"
sample_dir = "/root/annotated_textual_inversion/output/samples"
log_file = "/root/annotated_textual_inversion/output/losses.json"

with open(log_file, "w") as write_file:
    losses = lpips_validate_dir(loss_fn, sample_dir, target_dir)
    json.dump(losses, write_file)


In [19]:
# caculate the average loss
import json

log_file = "F:\\Work\\annotated_textual_inversion\\data\\loss\\Monet-TI-2000-losses.json"

sum_loss = 0
num_samples = 0

with open(log_file, "r") as read_file:
    losses = json.load(read_file)
    for key, value in losses.items():
        num_samples += len(value)
        sum_loss += sum(value)

print(f"average loss: {sum_loss / num_samples}")

average loss: 0.7249849343299866
